# Scraping

In [1]:
import sys

sys.path.append("..")
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import mlflow
import requests

pd.set_option('display.max_colwidth', None)  # For pandas versions < 1.0, use -1 instead of None

from dateutil import parser

from bs4 import BeautifulSoup
from datetime import datetime




warnings.filterwarnings('ignore')


### Load

In [2]:
df=pd.read_csv('d_drive_new.csv') # change name of original file

### Rename 2 features

In [3]:
df.rename(columns={'discover_clicks': 'external_clicks', 'discover_impressions': 'external_impressions'}, inplace=True)
df


,PAGE_EFAHRER_ID,DATE,PUBLISHED_AT,PUBLISH_DATE_EQUAL_TO_DATE,PAGE_CANONICAL_URL,PAGE_NAME,CLASSIFICATION_PRODUCT,CLASSIFICATION_TYPE,TITLE,PAGE_AUTHOR,DAILY_LIKES,DAILY_DISLIKES,WORD_COUNT,VIDEO_PLAY,IMPRESSIONS,DISCOVER_CLICKS,DISCOVER_IMPRESSIONS
0,1010803,2023-01-02,NaN,N,https://efahrer.chip.de/news/tariferhoehungen-und-thg-praemie-ladesaeulenbetreiber-ziehen-kunden-ueber-den-tisch_1010803,efa-1010803 | Tariferhöhungen und THG-Prämie: Ladesäulenbetreiber ziehen Kunden über den Tisch,THG,News,Tariferhöhungen und THG-Prämie: Ladesäulenbetreiber ziehen Kunden über den Tisch,Karl Lüdecke,NaN,NaN,NaN,1261.0,1375.0,1301.0,20323.0
1,1010592,2023-01-02,NaN,N,https://efahrer.chip.de/news/das-logo-von-alfa-romeo-wofuer-stehen-schlange-und-kreuz_1010592,efa-1010592 | Alfa Romeo: Was bedeuten Schlange und Kreuz?,Auto,News,Alfa Romeo: Was bedeuten Schlange und Kreuz?,Karl Müller,NaN,NaN,NaN,286.0,298.0,164.0,1493.0
2,1010719,2023-01-05,NaN,N,https://efahrer.chip.de/news/titel-ist-zurueck-in-deutschland-forscher-entwickeln-effizienteste-solarzelle_1010719,efa-1010719 | Rennen um die effizienteste Solarzelle: Deutsche holen sich den Titel zurück,Solaranlagen,News,Rennen um die effizienteste Solarzelle: Deutsche holen sich den Titel zurück,Aslan Berse,NaN,NaN,NaN,156.0,300.0,303.0,4912.0
3,1010727,2023-01-05,NaN,N,https://efahrer.chip.de/news/entlastungen-fuer-heizoel-kunden-wer-keinen-antrag-stellt-bekommt-gar-nichts_1010727,efa-1010727 | Antrag stellen oder leer ausgehen: Diese Entlastung ist für Heizöl-Kunden geplan,Energie,Ratgeber,Antrag stellen oder leer ausgehen: Diese Entlastung ist für Heizöl-Kunden geplan,CHIP,NaN,NaN,NaN,16.0,55.0,14009.0,92422.0
4,1010557,2023-01-02,2023-01-02,Y,https://efahrer.chip.de/news/solaranlage-auch-fuer-mieter-das-sind-die-besten-modelle-fuer-ihren-balkon_1010557,efa-1010557 | Balkonkraftwerk kaufen: Das sind die besten Modelle,Balkonkraftwerk,Kaufberatung,Balkonkraftwerk kaufen: Das sind die besten Modelle,Eva Goldschald,17.0,1.0,1513.0,174.0,128.0,6494.0,114984.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132841,1010297,2023-11-17,2023-11-17,Y,https://efahrer.chip.de/news/hammer-preis-fuer-hammer-auto-hyundai-ioniq-6-schon-fuer-200-euro-im-leasing_1010297,efa-1010297 | Hyundai IONIQ 6 Deal,E-Auto,Deal,Hyundai IONIQ 6 Deal,Lisa Brack,0.0,0.0,725.0,1.0,1.0,905.0,12463.0
132842,1014887,2023-12-31,2023-12-13,N,https://efahrer.chip.de/news/aldi-verkauft-rueckfahrkamera-fuers-auto-der-selbsteinbau-soll-ein-klacks-sein_1014887,efa-1014887 | Aldi verkauft Rückfahrkamera fürs Auto: Der Selbsteinbau soll ein Klacks sein,Zubehör,Deal,Aldi verkauft Rückfahrkamera fürs Auto: Der Selbsteinbau soll ein Klacks sein,Vanessa Finkler,0.0,0.0,535.0,0.0,1.0,95.0,1048.0
132843,105946,2024-01-18,2024-01-16,N,https://efahrer.chip.de/news/guenstiges-e-auto-trotz-praemien-aus-bei-diesen-neuwagen-sparen-sie-am-meisten_105946,"efa-105946 | E-Autos mit kurzer Lieferzeit: Fiat 500e, Polestar 2, Renault Twingo und mehr",E-Auto,Deal,"E-Autos mit kurzer Lieferzeit: Fiat 500e, Polestar 2, Renault Twingo und mehr",Benjamin Dizdar,0.0,0.0,2142.0,0.0,1.0,10.0,106.0
132844,1014127,2023-08-17,NaN,N,https://efahrer.chip.de/news/urlaubshoelle-in-italien-deutscher-strandet-mit-billigem-e-auto_1014127,efa-1014127 | Urlaubshölle in Italien: Deutsche stranden mit billigem E-Auto,E-Auto,News,Urlaubshölle in Italien: Deutsche stranden mit billigem E-Auto,Vanessa Finkler,NaN,NaN,NaN,0.0,1.0,5820.0,83900.0


### ...make everything lower case.


In [7]:
df.columns = df.columns.str.lower()

### Create a df  'scraping'.

In [8]:

scraping = df[['page_canonical_url','page_efahrer_id']].drop_duplicates(subset=['page_efahrer_id'])# this is a df 
                                        # made by 2 columns that drops all second identical rows of
                                        # 'page_efahrer_id'.                                 
scraping['current_title'] = 'dummy'
scraping['abstract'] = 'null'
scraping['author'] = 'null'
scraping['words'] = 'null'
scraping['last_update'] = 'null'
scraping = scraping.set_index('page_efahrer_id')


### A function to convert Date to Time Stamp.

In [9]:

def convert_date_to_timestamp(date_str):
   
    month_translation = {
        'Januar': 'January',
        'Februar': 'February',
        'März': 'March',
        'April': 'April',
        'Mai': 'May',
        'Juni': 'June',
        'Juli': 'July',
        'August': 'August',
        'September': 'September',
        'Oktober': 'October',
        'November': 'November',
        'Dezember': 'December'
    }


    for german, english in month_translation.items():
        if german in date_str:
            date_str = date_str.replace(german, english)
            break  

    try:# some strings of date are in strange fonts, so I try to make it and if doesnt work I just grab what is
        # there
        date_obj = datetime.strptime(date_str, '%d. %B %Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        # if conversion fails, return original date string 
        return date_str


In [12]:


# session object
session = requests.Session()
session.max_redirects = 5 #dont understand in depth this but when we set max redirects to 5 the code ran.

for index in scraping.index:
    url_for_index = scraping.loc[index, 'page_canonical_url']
    try:
        # this sends a request and builds below a 'soup' object with the parsed response
        response = session.get(url_for_index, allow_redirects=True, timeout=20)
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.find('h1').get_text() if soup.find('h1') else 'No Title Available'
        scraping.at[index, 'current_title'] = title

        abstract_element = soup.find(id='content').find('article').find('p') if soup.find(id='content') and soup.find(id='content').find('article') else None
        scraping.at[index, 'abstract'] = abstract_element.get_text(strip=True) if abstract_element and abstract_element.get_text(strip=True) else 'No Abstract Available'

        last_update_element = soup.find(id='content').find('article').find('h4').find('span') if soup.find(id='content') and soup.find(id='content').find('article') and soup.find(id='content').find('article').find('h4') else None
        scraping.at[index, 'last_update'] = convert_date_to_timestamp(last_update_element.text) if last_update_element else 'No Last Update Available'

        author_element = soup.find(id='content').find('article').find('h4').find('a') if soup.find(id='content') and soup.find(id='content').find('article') and soup.find(id='content').find('article').find('h4') else None
        scraping.at[index, 'author'] = author_element.text if author_element else 'No Author Available'

        image_url_element = soup.find('meta', attrs={'property': 'og:image'}) if soup.find('meta', attrs={'property': 'og:image'}) else None
        scraping.at[index, 'image_URL'] = image_url_element['content'] if image_url_element else 'No Image URL Available'

        page_text = soup.get_text(separator=' ', strip=True)
        word_count = len(page_text.split())
        scraping.at[index, 'words'] = word_count
        
        
    except requests.TooManyRedirects:
        print(f"Too many redirects for URL: {url_for_index}")
        continue  # Skip this URL and continue with the next iteration
    except requests.RequestException as e:
        print(f"Request failed for URL: {url_for_index}, Error: {e}")
        continue



In [13]:
scraping

,page_canonical_url,current_title,abstract,author,words,last_update,image_URL
page_efahrer_id,,,,,,,
1010803,https://efahrer.chip.de/news/tariferhoehungen-und-thg-praemie-ladesaeulenbetreiber-ziehen-kunden-ueber-den-tisch_1010803,"Neueste Förderung für E-Autos: Konzerne profitieren, Autofahrer zahlen",Die Preise für das öffentliche Laden von E-Autos steigen. Die Anbieter von Ladesäulen kassieren indes Millionen Euro Fördergelder.,Karl Lüdecke,1166,2023-08-18,https://im-efahrer.chip.de/files/enbw-schnelllader-63aae3726f96f.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=79ff9f23765b60904c084b7baab7e9141e0c9fef6a93ed0b10754b5d01778863
1010592,https://efahrer.chip.de/news/das-logo-von-alfa-romeo-wofuer-stehen-schlange-und-kreuz_1010592,Das Logo von Alfa Romeo: Wofür stehen Schlange und Kreuz?,Eine Schlange mit einem Menschen im Maul und ein rotes Kreuz machen das Logo von Alfa Romeo unverwechselbar. Italienreisende könnten es auch an einem anderen Ort gesehen haben. EFAHRER.com zeigt den Ursprung des Logos.,Karl Müller,1230,2022-12-25,https://im-efahrer.chip.de/files/alfa-romeo-lingyue-639c4b9530585c33298c1380-1-dec-21-2022-12-48-37-poster-63a305b458b42.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=e6524fc08d6cf3442d0760a6d298b9e539b42842d89c723f301344f408638d63
1010719,https://efahrer.chip.de/news/titel-ist-zurueck-in-deutschland-forscher-entwickeln-effizienteste-solarzelle_1010719,Titel ist zurück in Deutschland: Forscher entwickeln effizienteste Solarzelle,"Das Helmholtz-Zentrum Berlin holt den Titel für die effizienteste Tandemsolarzelle zurück nach Deutschland. Ein ForscherInnen-Team hat eine neue Silizium-Perowskit-Zelle entwickelt, die einen Wirkungsgrad von 32,5 Prozent erreicht",Aslan Berse,1103,2023-01-01,https://im-efahrer.chip.de/files/tandemglove2k-63a1b30666b39.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=fcde91bbda2af6f4576dfa185c69a59d4deb00be8da91107b8f04e0ccaa03848
1010727,https://efahrer.chip.de/news/entlastungen-fuer-heizoel-kunden-wer-keinen-antrag-stellt-bekommt-gar-nichts_1010727,"Entlastungen für Heizöl-Kunden: Wer keinen Antrag stellt, geht komplett leer aus","Es soll nicht nur für Gas, Fernwärme und Strom eine Entlastung geben, sondern auch für Haushalte, die Heizöl oder Pellets verwenden. Die Idee eines Härtefallfonds steht im Raum. Betroffene müssen jedoch Eigeninitiative zeigen und einige Hürden überwinden.",CHIP,1200,2023-01-03,https://im-efahrer.chip.de/files/heizunggeldstaat-63a2bf7a875a3.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=c4f92dd4d92acdd4f6bb495979dc3e9f75323cf5dbaa1c9a8fb538f706527ba4
1010557,https://efahrer.chip.de/news/solaranlage-auch-fuer-mieter-das-sind-die-besten-modelle-fuer-ihren-balkon_1010557,Balkonkraftwerk kaufen: Das sind die besten Modelle und Preise,"Wer kein Haus hat, kann trotzdem eineSolaranlage installieren. Die Lösung heißt Balkonkraftwerk. Diese Mini-Photovoltaikanlagen sind kostengünstig zu kaufen, sparen Stromkosten und sind schnell installiert. Wenn Sie ein Balkonkraftwerk kaufen möchten, finden Sie in unserem Artikel die wichtigsten Informationen und Empfehlungen zu leistungsstarken Modellen.",Eva Goldschald,4874,2024-01-06,https://im-efahrer.chip.de/files/balkonkraftwerk-100-t-1667393716198-v-16to9-retina-639349e541083.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=c8971dcfb1a850fa838ac2b8a493a6280f90f898a53ae9c146d2aef571e78c2a
...,...,...,...,...,...,...,...
1016319,https://efahrer.chip.de/news/elektro-beliebter-als-diesel-so-erfolgreich-sind-e-autos-heute-wirklich_1016319,dummy,null,null,null,null,NaN
1010895,https://efahrer.chip.de/news/nie-wieder-kabelsalat-am-taxistand-pfiffige-lade-loesung-fuer-koeln_1010895,dummy,null,null,null,null,NaN
1018743,https://efahrer.chip.de/news/irren-pfusch-am-e-auto-lader-kann-jeder-nachbauen-ist-dieser-hack-gefaehrlich_1018743,dummy,null,null,null,null,NaN
